In [1]:
# Disabled too many blank lines in python --> settings --> inspection --> python --> coding stlye PEP8 --> added code E303

import numpy as np          # dealing with arrays
import os                   # dealing with directories
import pandas as pd
from tqdm import tqdm       # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
import matplotlib.pyplot as plt
import pickle as pk
data_files_dir = '/home/moe/PycharmProjects/clean_code/Cleaned_code/data_files/'
testing_dir = '/home/moe/Desktop/bambo_Logs/Data/my_dataset/testing/'
validation_dir = '/home/moe/Desktop/bambo_Logs/Data/my_dataset/validation/'
training_dir = '/home/moe/Desktop/bambo_Logs/Data/my_dataset/training/'


HOME_DIR = '/home/moe/PycharmProjects/clean_code/'
elevator_path = '/home/moe/Desktop/bambo_Logs/Data/elevator_fulldata/process/'
excel_file = '/home/moe/Desktop/bambo_Logs/Data/weight_log_summary_602020.xlsx'
dataset_path = '/home/moe/Desktop/bambo_Logs/Data/my_dataset/'

excel_data = []
dfs_train = pd.read_excel(excel_file, sheet_name='All_training')
dfs_test = pd.read_excel(excel_file, sheet_name='All_testing')
dfs_validate = pd.read_excel(excel_file, sheet_name='All_validation')
excel_data.append(dfs_train.get_values())
excel_data.append(dfs_test.get_values())
excel_data.append(dfs_validate.get_values())
# print(len(excel_data[0]), len(excel_data[1]), len(excel_data[2]))

# excel_data[dataset][element_index][0 = weigh and 1= log_name]

elev_files_dir = sorted(os.listdir(elevator_path))
subsets_dir = sorted(os.listdir(dataset_path))


In [2]:
speed_inner = []
volume_inner = []
speed_complete = []
elevator_files = []
split = []
weight_arr = []
log_path_arr = []
img_path_arr = []
log_name_arr = []
speed_arr = []
volume_arr = []
full_data_dict = {}
full_data = []
log_length = []
missing_speed = []
missing_volume = []

random = False

for n, file in enumerate(elev_files_dir):
    file_ = file.split('.')[0]
    splity = file_.split('_')
    file_temp = ''
    for m in range(len(splity)):
        if m != 3:
            file_temp += splity[m]+'_'
    file_temp = file_temp.rstrip('_')
    elevator_files.append(file_temp)
log_ = ''

for i, subset in enumerate(subsets_dir):
    full_data_dict.clear()
    speed_arr.clear()
    volume_arr.clear()
    img_path_arr.clear()
    weight_arr.clear()
    log_path_arr.clear()
    log_name_arr.clear()
    log_length.clear()
    subset_name = subset
    subset_path = dataset_path + subset + '/'
    subset_dir = sorted(os.listdir(subset_path))
    if random:
        np.random.shuffle(subset_dir)
    # print(subset_dir)
    for j, log in enumerate(subset_dir):

        log_path = subset_path + log + '/'
        log_dir = sorted(os.listdir(log_path))
        log_temp = ''
        splity = log.split('_')
        weight = np.float32(splity[len(splity)-1])
        for m in range(len(splity)-1):
            if m != 3:
                log_temp += splity[m]+'_'
            log_ = log_temp.rstrip('_')
        for n, file in enumerate(elev_files_dir):
            file_ = elevator_files[n]
            file_path = elevator_path + file
            if file_ == log_:
                speed_inner.clear()
                volume_inner.clear()
                missing_speed.clear()
                missing_volume.clear()
                
                log_file = open(file_path, "r")
                for line in log_file:
                    split = line.split(" ")
                    speed = np.float32(split[6])
                    volume = np.float32(split[10])
                    speed_inner.append(speed)
                    volume_inner.append(volume)
                gap = len(log_dir) - len(speed_inner)
                for element in range(gap):
                    missing_speed.append(np.float32(0.0))
                    missing_volume.append(np.float32(0.0))
                speed_arr = speed_arr + missing_speed + speed_inner
                volume_arr = volume_arr + missing_volume + volume_inner
                for img in log_dir:
                    img_path = log_path + img
                    img_path_arr.append(img_path)
                    log_name_arr.append(log)
                    log_path_arr.append(log_path)
                    weight_arr.append(weight)
                    log_length.append(len(log_dir))

                break
    #    """
    #    This code saves data to a numpy array instead of pickle
    #
    # weight_arr_np = np.array(weight_arr)
    # log_name_arr_np = np.array(log_name_arr)
    # log_path_arr_np = np.array(log_path_arr)
    # speed_arr_np = np.array(speed_arr)
    # full_data_dict = {'speed': speed_arr_np, 'log_path': log_path_arr_np, 'log_name': log_name_arr_np, 'weight': weight_arr_np}
    # np.save(subset_name+ '_data.npy', full_data_dict)
    #
    #     """
    #     break   # This break to generate a single log
    if random:
        randomized = "_randomized_"
    else: randomized = "_"
    full_data_dict = {"volume": volume_arr, "speed": speed_arr, "log_path": log_path_arr, "img_path": img_path_arr, "log_name": log_name_arr,"log_length": 
        log_length, "weight": weight_arr}
    pickle_out = open(data_files_dir+subset_name+randomized+"dataset.pickle", "wb")
    pk.dump(full_data_dict, pickle_out)
    pickle_out.close()

In [3]:
# Test generation
import random

HOME_DIR = '/home/moe/PycharmProjects/clean_code/'
MASK_path = '/home/moe/Desktop/bambo_Logs/mask/mask_binary.bmp'

#  data shuffling is done through this, so there is no need to shuffle it in tensorflow
randomize = False
if randomize:
    randomized = "_randomized_"
else: 
    randomized = "_"
    
pickle_in_train = open(data_files_dir + "training"+randomized+"dataset.pickle", "rb")
training_data = pk.load(pickle_in_train)
pickle_in_train.close()


pickle_in_validate = open(data_files_dir + "validation"+randomized+"dataset.pickle", "rb")
validation_data = pk.load(pickle_in_validate)
pickle_in_validate.close()
pickle_in_test = open(data_files_dir + "testing"+randomized+"dataset.pickle", "rb")
testing_data = pk.load(pickle_in_test)
pickle_in_test.close()


training_weight = training_data['weight']
training_path = training_data['img_path']
training_speed = training_data['speed']
training_log_length = training_data['log_length']
training_volume = training_data['volume']


# img = plt.imread(training_path[0])


validation_weight = validation_data['weight']
validation_path = validation_data['img_path']
validation_speed = validation_data['speed']
validation_log_length = validation_data['log_length']
validation_volume = validation_data['volume']

testing_weight = testing_data['weight']
testing_path = testing_data['img_path']
testing_speed = testing_data['speed']
testing_log_length = testing_data['log_length']
testing_volume = testing_data['volume']

index = random.randint(1, len(validation_log_length))
# index = 1000

print(' ', training_path[index], ' ', training_weight[index], ' ', training_speed[index], ' ', training_log_length[index],  training_volume[index], 
      index,
      '\n\n')

print(len(training_log_length), len(testing_log_length), len(validation_log_length))

  /home/moe/Desktop/bambo_Logs/Data/my_dataset/training/2015_07_20_1746_bamb_41_609/609_0_3609.694-0.739352.bmp   609.0   0.73935   423 0.01221 19682 


70002 21297 26496
